In [ ]:
from lcms_processing_functions import *
from output_dataframes import *

def process_compound_data(file_path):
    """
    Reads an Excel file, processes each compound, and stores processed data in a dictionary.

    Parameters:
    - file_path (str): Path to the Excel file containing raw data.

    Returns:
    - dict: Dictionary with processed data for each compound, including calibration, concentrations, QC, and replicate statistics.
    """
    # Read the Excel file
    df = pd.read_excel(file_path)
    compound_dict = {}

    # Process each compound separately
    for compound, data in df.groupby('Compound'):
        compound_name = compound.split('_')[-1]
        
        # Perform calibration and calculate concentrations
        cal_std_df, LOQ = process_calibration(data)
        cal_curve = get_calibration_curve(cal_std_df, LOQ)
        samples_df = process_samples(data)
        cal_std_df = calculate_concentrations(cal_std_df, LOQ, cal_curve)
        samples_df = calculate_concentrations(samples_df, LOQ, cal_curve)
        
        # Modify 'Sample Type' for categorization
        samples_df = sample_type(samples_df)
        
        # Perform blank subtraction
        samples_df = subtraction(samples_df)

        # Perform QC calculations
        dup_precision_df = duplication_comparison(samples_df)
        spike_precision_df = spike_comparison(samples_df)
        # qc_precision_df = qc_comparison(samples_df, cal_std_df)

        # Calculate replicate statistics
        replicate_stats_df = replicate_stats(samples_df)

        # Store all processed data in the compound dictionary
        compound_dict[compound_name] = {
            'Cal Std': cal_std_df,
            'Sample': samples_df,
            'LOQ': LOQ,
            'Regression results': cal_curve,
            'Duplication Precision': dup_precision_df,
            'Spike Precision': spike_precision_df,
            # 'QC Precision': qc_precision_df,
            'Replicate Statistics': replicate_stats_df
        }

    return compound_dict

def filter_compounds(compound_dict):

    """
    Filters out isotopically-labelled compounds from the compound dictionary based on their names.
    Excludes compounds containing '13C2', starting with 'M' or 'd' followed by a digit,
    or matching 'PFHxS(O18)2'.

    Parameters:
    - compound_dict (dict): A dictionary of compounds with associated data.

    Returns:
    - dict: A dictionary containing only the filtered compounds.
    """
    filtered_compounds = {
        compound: data
        for compound, data in compound_dict.items()
        if not (
            ("13C2" in compound) or
            (compound.startswith("M") and compound[1].isdigit()) or
            (compound.startswith("d") and compound[1].isdigit()) or
            (compound == "PFHxS(O18)2")
        )
    }

    return filtered_compounds

In [ ]:
#### Insert your input Excel spreadsheet filepath below ####
file_path = './20250110_Alaska_Community_LLE.xlsx'
compound_dict = process_compound_data(file_path)

In [ ]:
#### Insert your output Excel spreadsheet filepath below ####
output_filename = "../../Alaskan_Community_LLE_Summary.xlsx"

filtered_compounds = filter_compounds(compound_dict)
# Generate DataFrames
df_loq = create_loq_dataframe(filtered_compounds)
df_smp_concentrations = create_smp_concentrations_dataframe(filtered_compounds)
df_duplication = create_dup_prec_dataframe(filtered_compounds)
df_spike = create_spike_prec_dataframe(filtered_compounds)
df_qc = create_qc_prec_dataframe(filtered_compounds)

with pd.ExcelWriter(output_filename, engine="xlsxwriter") as writer:
    workbook = writer.book

    # Define the header format
    header_format = workbook.add_format({
        'bold': True,
        'bg_color': '#C1E1FF',
        'border': 1
    })

    # Write each sheet and format the first row headers
    if not df_loq.empty:
        df_loq.to_excel(writer, sheet_name="LOQ", index=False, header=False, startrow=1)  # Leave space for custom headers
        worksheet = writer.sheets["LOQ"]
        # Write and format headers manually
        for col_num, value in enumerate(df_loq.columns):
            worksheet.write(0, col_num, value, header_format)  # Write header with formatting

    if not df_smp_concentrations.empty:
        df_smp_concentrations.to_excel(writer, sheet_name="Concentrations", index=False, header=False, startrow=1)
        worksheet = writer.sheets["Concentrations"]
        for col_num, value in enumerate(df_smp_concentrations.columns):
            worksheet.write(0, col_num, value, header_format)

    if not df_duplication.empty:
        df_duplication.to_excel(writer, sheet_name="Duplication Precision", index=False, header=False, startrow=1)
        worksheet = writer.sheets["Duplication Precision"]
        for col_num, value in enumerate(df_duplication.columns):
            worksheet.write(0, col_num, value, header_format)

    if not df_spike.empty:
        df_spike.to_excel(writer, sheet_name="Spike Precision", index=False, header=False, startrow=1)
        worksheet = writer.sheets["Spike Precision"]
        for col_num, value in enumerate(df_spike.columns):
            worksheet.write(0, col_num, value, header_format)

    if not df_qc.empty:
        df_qc.to_excel(writer, sheet_name="QC Precision", index=False, header=False, startrow=1)
        worksheet = writer.sheets["QC Precision"]
        for col_num, value in enumerate(df_qc.columns):
            worksheet.write(0, col_num, value, header_format)

print(f"Excel file saved to '{output_filename}'")
